In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard fbe67e465375231474a2ad80a4389efc77ecff99

Cloning into 'yolov5'...
remote: Enumerating objects: 15305, done.
remote: Total 15305 (delta 0), reused 0 (delta 0), pack-reused 15305
Receiving objects: 100% (15305/15305), 14.19 MiB | 16.23 MiB/s, done.
Resolving deltas: 100% (10500/10500), done.
/content/yolov5
HEAD is now at fbe67e4 Fix `OMP_NUM_THREADS=1` for macOS (#8624)


In [ ]:
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 62.7 MB/s eta 0:00:00
Setup complete. Using torch 1.13.1+cu116 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15101MB, multi_processor_count=40)


In [ ]:
%cd /content/yolov5
#follow the link below to get your download code from from Roboflow
!pip install roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="TBSZIcJvZGLN6tqEEfeO")
project = rf.workspace("tfnet-night-vision").project("mul")
dataset = project.version(2).download("yolov5")
%cat {dataset.location}/data.yaml

/content/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 KB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 KB 22.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9676 sha256=9ca7fee9a275cae2e92519e9d0beedde1719bb966193a13dd688c0fca8b203c9
  Stored in directory: /root/.cache/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.0.9
    Uninstalling pyparsing-3.0.9:
      Successfully uninstalled pyparsing-3.0.9
  Att

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Mul-2 in yolov5pytorch:: 100%|██████████| 16733/16733 [00:03<00:00, 4868.40it/s]


names:
- Bird
- Drone
- Helicopter
- Plane
nc: 4
roboflow:
  license: CC BY 4.0
  project: mul
  url: https://universe.roboflow.com/tfnet-night-vision/mul/dataset/2
  version: 2
  workspace: tfnet-night-vision
test: ../test/images
train: Mul-2/train/images
val: Mul-2/valid/images


In [ ]:
# define number of classes based on YAML
import yaml
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
#this is the model configuration we will use for our tutorial 
%cat //content/yolov5/models/yolov5s.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]]

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate /content/yolov5/models/YOLOG-p345.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors: 3  # AutoAnchor evolves 3 anchors per P output layer
AutoBatch:
#anchors:
#  - [10,13, 16,30, 33,23]  # P3/8 small
#  - [30,61, 62,45, 59,119]  # P4/16 medium
# - [116,90, 156,198, 373,326]  # P5/32 large

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, GhostConv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3Ghost, [128]],
   [-1, 1, GhostConv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3Ghost, [256]],
   [-1, 1, GhostConv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3Ghost, [512]],
   [-1, 1, GhostConv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3Ghost, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, GhostConv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3Ghost, [512, False]],  # 13

   [-1, 1, GhostConv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3Ghost, [256, False]],  # 17 (P3/8-small)

   [-1, 1, GhostConv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3Ghost, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, GhostConv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3Ghost, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
# time its performance
#%%time
#%cd /content/yolov5/
#Adds YOLOv5 autobatch feature to solve for best batch size on CUDA devices by passing --batch-size -1 to train.py:
#!python train.py --img 320 --batch-size -1 --epochs 600 --optimizer AdamW --data {dataset.location}/data.yaml --cfg ./models/YOLOG-p345.yaml --hyp data/hyps/hyp.scratch-high.yaml --weights '' --project "/content/gdrive/MyDrive/Runs"  --name YOLOG-p345_results  --cache

/content/yolov5
train: weights=, cfg=./models/YOLOG-p345.yaml, data=/content/yolov5/Mul-2/data.yaml, hyp=data/hyps/hyp.scratch-high.yaml, epochs=600, batch_size=-1, imgsz=320, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=AdamW, sync_bn=False, workers=8, project=/content/gdrive/MyDrive/Runs, name=YOLOG-p345_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 417 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.

In [ ]:
#Later, if it is stopped prematurely and I need to resume, I use
%%time
%cd /content/yolov5/
!python train.py --img 320 --weights '' --project "/content/gdrive/MyDrive/Runs"  --name YOLOG-p345_results  --cache  --resume "/content/gdrive/MyDrive/Runs/YOLOG-p345_results/weights/last.pt"

/content/yolov5
train: weights=, cfg=, data=data/coco128.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=300, batch_size=16, imgsz=320, rect=False, resume=/content/gdrive/MyDrive/Runs/YOLOG-p345_results/weights/last.pt, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=/content/gdrive/MyDrive/Runs, name=YOLOG-p345_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 417 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
Resuming training from /content/gdrive/MyDrive/Runs/YOLOG-p345_results/weights/last.pt
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperp

In [ ]:
%cd /content/yolov5/ 
!python detect.py --weights /content/gdrive/MyDrive/Runs/YOLOG-p345_results/weights/best.pt  --img 320 --conf 0.4 --source /content/yolov5/Mul-2/valid/images 
#--visualize

/content/yolov5
detect: weights=['/content/gdrive/MyDrive/Runs/YOLOG-p345_results/weights/best.pt'], source=/content/yolov5/Mul-2/valid/images, data=data/coco128.yaml, imgsz=[320, 320], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 366 layers, 3683817 parameters, 0 gradients, 8.1 GFLOPs
image 1/240 /content/yolov5/Mul-2/valid/images/00000000_0000000000916B52_w320_h256_pMono14_jpg.rf.91cf4077ddea0550081db9407290523f.jpg: 320x320 1 Plane, Done. (0.009s)
image 2/240 /content/yolov5/Mul-2/valid/images/00000000_0000000001247F81_w320_h256_pMono14_jpg.rf.dfd5fb3f2ca4aaea319b0e564c870119

In [ ]:
%cd /content/yolov5/ 
!python detect.py --weights /content/gdrive/MyDrive/Runs/YOLOG-p345_results/weights/best.pt  --img 320 --conf 0.4 --source /content/yolov5/Mul-2/train/images 
#--visualize

Streaming output truncated to the last 5000 lines.
image 3125/8122 /content/yolov5/Mul-2/train/images/2022-10-13-84-_png_jpg.rf.497434a7a867689bef8e9bd17c3d92d4.jpg: 320x320 1 Helicopter, Done. (0.013s)
image 3126/8122 /content/yolov5/Mul-2/train/images/2022-10-13-84-_png_jpg.rf.5120f6fb78895c6a2a027ae24d6d391e.jpg: 320x320 1 Helicopter, Done. (0.009s)
image 3127/8122 /content/yolov5/Mul-2/train/images/2022-10-13-84-_png_jpg.rf.60c383ad544b2bebcdcad2f5658386e5.jpg: 320x320 1 Helicopter, Done. (0.009s)
image 3128/8122 /content/yolov5/Mul-2/train/images/2022-10-13-84-_png_jpg.rf.6d9cc2fdda6ed72abe86c34533103997.jpg: 320x320 1 Helicopter, Done. (0.009s)
image 3129/8122 /content/yolov5/Mul-2/train/images/2022-10-13-84-_png_jpg.rf.76cc1f2f784e7674447787e0120863a4.jpg: 320x320 1 Helicopter, Done. (0.009s)
image 3130/8122 /content/yolov5/Mul-2/train/images/2022-10-13-84-_png_jpg.rf.7a3d5a7fbf1851a6becf4f21b77b018f.jpg: 320x320 1 Helicopter, Done. (0.009s)
image 3131/8122 /content/yolov5/Mul-2

In [ ]:
!python val.py --task study --weights /content/gdrive/MyDrive/Runs/YOLOG-p345_results/weights/best.pt --data {dataset.location}/data.yaml --iou 0.5 --save-txt

val: data=/content/yolov5/Mul-2/data.yaml, weights=['/content/gdrive/MyDrive/Runs/YOLOG-p345_results/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.5, task=study, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=True, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False

Running study_data_best.txt --imgsz 256...
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 366 layers, 3683817 parameters, 0 gradients, 8.1 GFLOPs
100% 755k/755k [00:00<00:00, 16.2MB/s]
val: Scanning '/content/yolov5/Mul-2/valid/labels' images and labels...240 found, 0 missing, 0 empty, 0 corrupt: 100% 240/240 [00:00<00:00, 887.61it/s]
val: New cache created: /content/yolov5/Mul-2/valid/labels.cache
               Class     Images     Labels         TP         FP          P          R         F1     mAP@.5 mAP@.5:.95: 100% 8/8

In [ ]:
!python val.py --task speed --weights /content/gdrive/MyDrive/Runs/YOLOG-p345_results/weights/best.pt --data {dataset.location}/data.yaml --iou 0.5 --save-txt

val: data=/content/yolov5/Mul-2/data.yaml, weights=['/content/gdrive/MyDrive/Runs/YOLOG-p345_results/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.5, task=speed, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=True, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 366 layers, 3683817 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning '/content/yolov5/Mul-2/valid/labels.cache' images and labels... 240 found, 0 missing, 0 empty, 0 corrupt: 100% 240/240 [00:00<?, ?it/s]
               Class     Images     Labels         TP         FP          P          R         F1     mAP@.5 mAP@.5:.95: 100% 8/8 [00:02<00:00,  2.96it/s]
                 all        240        269       66.2       1.75      0.974      0.985      0.979      0.988      0.684
    

In [ ]:
!python val.py --img 320 --weights /content/gdrive/MyDrive/Runs/YOLOG-p345_results/weights/best.pt --data {dataset.location}/data.yaml --iou 0.5 --verbose --save-txt

val: data=/content/yolov5/Mul-2/data.yaml, weights=['/content/gdrive/MyDrive/Runs/YOLOG-p345_results/weights/best.pt'], batch_size=32, imgsz=320, conf_thres=0.001, iou_thres=0.5, task=val, device=, workers=8, single_cls=False, augment=False, verbose=True, save_txt=True, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 366 layers, 3683817 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning '/content/yolov5/Mul-2/valid/labels.cache' images and labels... 240 found, 0 missing, 0 empty, 0 corrupt: 100% 240/240 [00:00<?, ?it/s]
               Class     Images     Labels         TP         FP          P          R         F1     mAP@.5 mAP@.5:.95: 100% 8/8 [00:05<00:00,  1.41it/s]
                 all        240        269       65.8          1      0.981       0.98      0.981       0.98      0.693
       

In [ ]:
#### Visulize each layers feature map
#######Run this cell to know the whats inside every lare feature
#### stop it ur self after one image is done
## Print whole folder feature maps and save them
#!python detect.py --weights /content/yolov5/mfnets.pt --img 416 --conf 0.4 --source /content/yolov5/Uav-7/test/images --visualize
####### Only single image
#!python detect.py --weights runs/train/GAANet-p2345_results/weights/best.pt --img 416 --conf 0.4 --source /content/yolov5/Uav-7/test/images/0094_jpg.rf.09856e96b7f9f77c2bc502452ec20d77.jpg --visualize                             

In [ ]:
##I have created a zip file:
#!zip -r /content/YOLOG-p345.zip /content/gdrive/MyDrive/Runs/
!zip -r /content/train_YOLOp345.zip /content/yolov5/runs/detect/exp
##Than I have downloded that zip file:
from google.colab import files
files.download("/content/train_YOLOp345.zip")

Streaming output truncated to the last 5000 lines.
  adding: content/yolov5/runs/detect/exp/scene02550_jpg.rf.193122142b7b0c72345c9bf79d3da011.jpg (deflated 5%)
  adding: content/yolov5/runs/detect/exp/2022-10-13-89-_png_jpg.rf.a7aa927853a22a0971b3ce311c86efe3.jpg (deflated 5%)
  adding: content/yolov5/runs/detect/exp/IR_DRONE_002-0080_jpg.rf.4055b2df5c558bcd693b49f9019506ad.jpg (deflated 6%)
  adding: content/yolov5/runs/detect/exp/scene00649_jpg.rf.404f8f1b680a7064342491eb8bc12624.jpg (deflated 5%)
  adding: content/yolov5/runs/detect/exp/scene00938_jpg.rf.6c57d01fb020b6dafdb2634897471834.jpg (deflated 4%)
  adding: content/yolov5/runs/detect/exp/IR_DRONE_001-0082_jpg.rf.e39c9a22cf36f0590d604c9471bbc86b.jpg (deflated 5%)
  adding: content/yolov5/runs/detect/exp/thermal1_0225_png_jpg.rf.089a4baaa5963b02950aa8319111b6b3.jpg (deflated 6%)
  adding: content/yolov5/runs/detect/exp/2022-10-13-111-_png_jpg.rf.341d2204cbd3e2db8ba2053b2bb21845.jpg (deflated 3%)
  adding: content/yolov5/runs/d

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
%cd /content/yolov5/ 
!python detect.py --weights /content/gdrive/MyDrive/Runs/YOLOG-p345_results/weights/best.pt --img 320 --conf 0.4 --source /content/yolov5/test1.jpg --save

/content/yolov5
usage: detect.py
       [-h]
       [--weights WEIGHTS [WEIGHTS ...]]
       [--source SOURCE]
       [--data DATA]
       [--imgsz IMGSZ [IMGSZ ...]]
       [--conf-thres CONF_THRES]
       [--iou-thres IOU_THRES]
       [--max-det MAX_DET]
       [--device DEVICE]
       [--view-img]
       [--save-txt]
       [--save-conf]
       [--save-crop]
       [--nosave]
       [--classes CLASSES [CLASSES ...]]
       [--agnostic-nms]
       [--augment]
       [--visualize]
       [--update]
       [--project PROJECT]
       [--name NAME]
       [--exist-ok]
       [--line-thickness LINE_THICKNESS]
       [--hide-labels]
       [--hide-conf]
       [--half]
       [--dnn]
detect.py: error: ambiguous option: --save could match --save-txt, --save-conf, --save-crop
